In [ ]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs

!ls german-traffic-signs

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import pickle
import pandas as pd
import random

np.random.seed(0)


In [ ]:
with open("german-traffic-signs/train.p", "rb") as f:
  train_data = pickle.load(f)
with open("german-traffic-signs/valid.p", "rb") as f:
  val_data = pickle.load(f)
with open("german-traffic-signs/test.p", "rb") as f:
  test_data = pickle.load(f)

X_train, y_train = train_data["features"], train_data["labels"]
X_val, y_val = val_data["features"], val_data["labels"]
X_test, y_test = test_data["features"], test_data["labels"]

# print(type(train_data))
# print(train_data["features"])
# print(train_data["labels"])
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

assert X_train.shape[0] == y_train.shape[0], "number of image is not equal to the number of labels"
assert X_val.shape[0] == y_val.shape[0], "number of image is not equal to the number of labels"
assert X_test.shape[0] == y_test.shape[0], "number of image is not equal to the number of labels"
assert X_train.shape[1:] == (32, 32, 3), "dimension of the images are not 32 x 32 x 3"
assert X_val.shape[1:] == (32, 32, 3), "dimension of the images are not 32 x 32 x 3"
assert X_test.shape[1:] == (32, 32, 3), "dimension of the images are not 32 x 32 x 3"

In [ ]:
data = pd.read_csv("german-traffic-signs/signnames.csv")

num_of_samples = []
 
cols = 5
num_classes = 43
 
fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(5, 50))
fig.tight_layout()
for i in range(cols):
    for j, row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, (len(x_selected) - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + "-" + row["SignName"])
            num_of_samples.append(len(x_selected))

In [ ]:
print(num_of_samples)
plt.figure(figsize=(18, 7))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

In [ ]:
import cv2

plt.imshow(X_train[1050])
plt.axis("off")
print(y_train[1050])

"""
Image preprocessing step 1 : Convert to grayscale
"""

def grayscale(img):
  return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = cv2.bitwise_not(grayscale(X_train[1000]))
plt.imshow(img, cmap=plt.get_cmap("gray"))
plt.axis("off")
print(img.shape)

"""
Image preprocessing step 2 : Apply equalisation histogram
"""

def equalise(img):
  return cv2.equalizeHist(img)    #this function works best with grayscale image

img = equalise(img)
plt.imshow(img, cmap=plt.get_cmap("gray"))
plt.axis("off")
print(img.shape)

In [ ]:
""" 
Applying steps over entire dataset
"""

def preprocessing(img):
  img = grayscale(img)
  img = equalise(img)
  img = img/255   #normalisation
  return img

X_train = np.array(list(map(preprocessing, X_train)))
X_val = np.array(list(map(preprocessing, X_val)))
X_test = np.array(list(map(preprocessing, X_test)))

# plt.imshow(X_train[500], cmap=plt.get_cmap("gray"))
# plt.axis("off")

In [ ]:
#reshaping 
X_train = X_train.reshape(34799, 32, 32, 1)
X_val = X_val.reshape(4410, 32, 32, 1)
X_test = X_test.reshape(12630, 32, 32, 1)


In [ ]:
"""
applying data augmentation over the training set
"""

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10)
datagen.fit(X_train)

In [ ]:
#printing augmented samples

batches = datagen.flow(X_train, y_train, batch_size=20)
X_batch, y_batch = next(batches)

fig, axs = plt.subplots(1, 15, figsize=(20, 5))
fig.tight_layout()

for i in range(15):
  axs[i].imshow(X_batch[i].reshape(32, 32), cmap=plt.get_cmap("gray"))
  axs[i].axis("off")

In [ ]:
#one-hot-encoding labels
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
"""
Defining a modified model to fit the data
"""

def modified_model():
  model = Sequential()

  #feature learning
  model.add(Conv2D(60, (5, 5), input_shape= (32, 32, 1), activation="relu")) 
  model.add(Conv2D(60, (5, 5), activation="relu"))    
  model.add(MaxPooling2D(pool_size= (2, 2)))  

  model.add(Conv2D(30, (3, 3), activation= "relu"))
  model.add(Conv2D(30, (3, 3), activation= "relu"))     
  model.add(MaxPooling2D(pool_size= (2, 2)))    
  
  # model.add(Dropout(0.5))    #dropout layer
  
  #Classification
  model.add(Flatten())  
  model.add(Dense(500, activation= "relu"))    #hidden layer 
  model.add(Dropout(0.5))    #dropout layer
  model.add(Dense(num_classes, activation= "softmax"))    #output layer
  
  model.compile(Adam(lr = 0.001), loss="categorical_crossentropy", metrics= ["accuracy"])
  return model

model = modified_model()
print(model.summary())

In [ ]:
batch_size = 50
steps_per_epoch = X_train.shape[0]/batch_size
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=steps_per_epoch, epochs=10, validation_data=(X_val, y_val), shuffle=1)

In [ ]:
plt.plot(history.history["accuracy"])    
plt.plot(history.history["val_accuracy"])   
plt.legend(["accuracy", "val_accuracy"])
plt.title("Accuracy")
plt.xlabel("epochs")

In [ ]:
plt.plot(history.history["loss"])     
plt.plot(history.history["val_loss"]) 
plt.legend(["loss", "val_loss"])
plt.title("Loss")
plt.xlabel("epochs")

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(type(score))
print("TEST SCORE : ", score[0])
print("TEST ACCURACY : ", score[1])

In [ ]:
#importing new image
import requests
from PIL import Image     #Python image library

url = "https://c8.alamy.com/comp/J2MRAJ/german-road-sign-bicycles-crossing-J2MRAJ.jpg"
response = requests.get(url, stream=True)
img = Image.open(response.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
import cv2

img_array = np.asarray(img)     #converts the input to array 
print(img_array.shape)
img_resized = cv2.resize(img_array, (32, 32))
image = preprocessing(img_resized)
plt.imshow(image, cmap = plt.get_cmap("gray"))
print(image.shape)

In [ ]:
image = image.reshape(1, 32, 32, 1)
print(image.shape)

In [ ]:
""" prediction """
prediction = np.argmax(model.predict(image), axis=-1)
print("Predicted Digit : ", prediction)